# Loading The Content and Style Images

Enter your code below...

# Leveraging the VGG16 Model

As we coded in the previous notebook, we will once again import the VGG architecture. However, this time we will only leverage the architecture and not need to load the weights. We will be creating our own loss chain to exploit VGG for Style Transfer.

Enter your code below...

# Creating the Content Loss

We will hand code the first loss function to be added to our loss chain by hand. Then we will use some pre-coded helper functions for the rest of the loss chain.

Enter your code below...


# \* HELPER FUNCTIONS \*

We're going to put the helper functions directly in the notebook this time instead of hidding them away. You don't need to be able to understand the math right now, but opening the hood and just looking should now not be so terrifying. 

The original Style Transfer notebook used in DLFD broke with recent Keras updates, so this notebook is almost completely based on the fantastic tutorials and notebooks found here: https://harishnarayanan.org/writing/artistic-style-transfer/ and here: https://github.com/hnarayanan/artistic-style-transfer. Make sure to check them out!

**NOTE**: Run the cell below to load in the helper functions, then continue your in class code in the next empty cell...


In [14]:
def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = height * width
    return backend.sum(backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

def total_variation_loss(x):
    a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))

def eval_loss_and_grads(x):
    x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


# *Continue Your Code Below...*